# Load & Prepare data

Give pointers and examples on how we prepared the data and computed S & I from large datasets.

In [3]:
# import
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import gmpy2
from tqdm import tqdm_notebook as tqdm
import scipy
import scipy.stats as st
import scipy.special as sp
from itertools import count, product
import importlib
import mpmath
from multiprocessing import Pool
import matplotlib as mpl
from glob import glob
import olga.load_model as load_model
import olga.generation_probability as pgen

mpl.rcParams['figure.dpi']= 300

# useful functions

# really basic way of parallelizing any operation on a dataframe
def parallelize_dataframe(df, func, args=None, n_cores=4):
    """
    Arguments:
    - df: dataframe to modify
    - func: function that accept a dataframe as its first argument and return a dataframe
    - args: additional arguments of fun
    - n_core: number of cores to use
    """
    df_split = np.array_split(df, n_cores)
    if args is None:
        all_args = [(d,) for d in df_split]
    else:
        all_args = [(d, *ags) for d, ags 
                    in zip(df_split, [args]*n_cores)]
    pool = Pool(n_cores)
    df = pd.concat(pool.starmap(func, all_args))
    pool.close()
    pool.join()
    return df

def compute_pgen_nt(df, pgen_model):
    df["pgen_nt"] = df.ntCDR3.apply(pgen_model.compute_nt_CDR3_pgen)
    return df

def load_olga_model(model_directory="olga3/olga/default_models/human_T_beta/"):
    """ Fully load an olga model """
    params_file_name = f'{model_directory}model_params.txt'
    marginals_file_name = f'{model_directory}model_marginals.txt'
    V_anchor_pos_file = f'{model_directory}V_gene_CDR3_anchors.csv'
    J_anchor_pos_file = f'{model_directory}J_gene_CDR3_anchors.csv'

    genomic_data = load_model.GenomicDataVDJ()
    genomic_data.load_igor_genomic_data(params_file_name, V_anchor_pos_file, J_anchor_pos_file)

    generative_model = load_model.GenerativeModelVDJ()
    generative_model.load_and_process_igor_model(marginals_file_name)

    pgen_model = pgen.GenerationProbabilityVDJ(generative_model, genomic_data)

    return genomic_data, generative_model, pgen_model


def load_olga_model_VJ(model_directory="olga3/olga/default_models/human_T_beta/"):
    """ Fully load an olga model """
    params_file_name = f'{model_directory}model_params.txt'
    marginals_file_name = f'{model_directory}model_marginals.txt'
    V_anchor_pos_file = f'{model_directory}V_gene_CDR3_anchors.csv'
    J_anchor_pos_file = f'{model_directory}J_gene_CDR3_anchors.csv'

    genomic_data = load_model.GenomicDataVJ()
    genomic_data.load_igor_genomic_data(params_file_name, V_anchor_pos_file, J_anchor_pos_file)

    generative_model = load_model.GenerativeModelVJ()
    generative_model.load_and_process_igor_model(marginals_file_name)

    pgen_model = pgen.GenerationProbabilityVJ(generative_model, genomic_data)

    return genomic_data, generative_model, pgen_model

def find_nt_seq(long_nt, aa):
    """Extract from the long_nt nucleotide sequence the first part that match the amino-acid sequence"""
    for ii in range(3):
        seq = nt_to_aa(long_nt[ii:])
        res = seq.find(aa)
        if res != -1:
            return long_nt[res*3 + ii]

def estimate_S_unbiased(df1, df2=None, M=4000):
    """
    Compute an estimate of S
    @Arguments
    - df1 & df2 are two dataframes that contains a column "count" (number of reads associated with each clonotype)
        & a column "ntCDR3" (with no doublons)
    - If df2 is None, compute the estimated sharing between "numerical" replicates
    - M is the size of the two samples (M = M_1 = M_2)
    """
    
    if df2 is None:
        Q = int(df1["count"].sum())
        def h(x):
            return (1 - 2*gmpy2.comb(Q-M, x)/gmpy2.comb(Q, x) 
                    + gmpy2.comb(Q-2*M, x)/gmpy2.comb(Q, x))
        # apply h on the column "count"
        return float(np.sum(df1["count"].apply(lambda x: h(int(x)))))
    else:
        Q1 = int(df1["count"].sum())
        Q2 = int(df2["count"].sum())
        def h(x1, x2):
            return ((1 - gmpy2.comb(Q1-M, x1)/gmpy2.comb(Q1, x1)) 
                    * (1 - gmpy2.comb(Q2-M, x2)/gmpy2.comb(Q2, x2)))
        # only keep shared sequences
        df = df1.merge(df2, how='inner', on="ntCDR3")
        return float(np.sum(
            df.apply(lambda x: h(int(x["count_x"]), int(x["count_y"])), 
                     axis=1)))
    
def estimate_I_unbiased(df1, df2=None, M=4000, gamma=12):
    """
    Compute an estimate of I
    @Arguments
    - df1 & df2 are two dataframes that contains a column "count" (number of reads associated with each clonotype)
        a column pgen_nt & a column "ntCDR3" (with no doublons)
    - If df2 is None, compute the estimated sharing between "numerical" replicates
    - M is the size of the two samples (M = M_1 = M_2)
    """
    
    if df2 is None:
        Q = int(df1["count"].sum())
        def h(x):
            return (1 - 2*gmpy2.comb(Q-M, x)/gmpy2.comb(Q, x) 
                    + gmpy2.comb(Q-2*M, x)/gmpy2.comb(Q, x))
        # apply h on the column "count"
        return float(np.sum(df1["count"].apply(lambda x: h(int(x))) * df1["pgen_nt"]))
    else:
        Q1 = int(df1["count"].sum())
        Q2 = int(df2["count"].sum())
        def h(x1, x2):
            return ((1 - gmpy2.comb(Q1-M, x1)/gmpy2.comb(Q1, x1)) 
                    * (1 - gmpy2.comb(Q2-M, x2)/gmpy2.comb(Q2, x2)))
        # only keep shared sequences
        df = df1.merge(df2, how='inner', on="ntCDR3")
        return float(np.sum(
            df.apply(lambda x: h(int(x["count_x"]), int(x["count_y"])), 
                     axis=1)  * (-np.log(df["pgen_nt_x"] - gamma))))
    

## Yellow Fever dataset

Data is associated with [Pogorely et. al](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6294963/), and is extracted from the SRA, accession number `PRJNA493983`. The dataset contains RepSeq data on three pairs of twins, with multiple time points both pre and post Yellow Fever immunization. We load already pre-processed data (migec + mixcr). Data is cDNA, and the library is generated through 5'-race. 

## Robins dataset

Data is associated with [Longitudinal immunosequencing in healthy people](https://pubmed.ncbi.nlm.nih.gov/31226930/), the dataset can be downloaded from [ImmuneAccess](https://clients.adaptivebiotech.com/immuneaccess) (adaptive biotechnologies)

## Emerson dataset

Data is associated with [Emerson et al.](https://www.nature.com/articles/ng.3822) and can be downloaded from [ImmuneAccess](https://clients.adaptivebiotech.com/immuneaccess) (adaptive biotechnologies).

## Example

Due to their sizes, only one repertoire (from the Yellow Fever Dataset) is attached. The code can take some time (it needs to compute a fair amount of generation probabilities).

In [4]:
# compute pgen on a dataset (YFV)
# code rely on olga (https://github.com/statbiophys/OLGA/tree/master/olga), 
# the default models folder is also present in the olga github
_, _, pgen_model = load_olga_model(
    model_directory="../datasets/default_models/human_T_beta/")

df = pd.read_csv(f"../datasets/S1_0_F1_.txt.gz", sep="\t", 
                 usecols=["Clone count", "Clone fraction", "N. Seq. CDR3"])
df.columns = ["count", "frequency", "ntCDR3"]
df = df.dropna(subset=["ntCDR3"]) # remove sequences where the CDR3 wasn't found
df = df[df.ntCDR3.apply(lambda x: len(x)%3 == 0)] # only keep the in-frame
# df = parallelize_dataframe(df, compute_pgen_nt, args=(pgen_model,), n_cores=4)

# estimate the (autologous) mean of S (M=5000) 
S = estimate_S_unbiased(df, df, M=5000)
print("S: ", S)

# estimate the (autologous) mean of I (M=5000) 
# I = estimate_I_unbiased(df, df, M=5000)
# print("I: ", I)

S:  199.7092593568384
